This is some connect 

In [238]:
# Must be included at the beginning of each new notebook. Remember to change the app name.
import findspark
findspark.init('/home/ubuntu/spark-3.2.1-bin-hadoop2.7')
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('basics').getOrCreate()

In [239]:
file = 'violence_data_NEW.csv'
df = spark.read.csv(file, header=True, inferSchema=True)

In [240]:
from pyspark.sql.types import (StructField,StringType,IntegerType,StructType)

In [241]:
#3.1 drop the 'Usere feedback' field
columns_to_drop = ['User feedback']
new_df = df.drop(*columns_to_drop)

In [242]:
new_df.columns

['RecordID',
 'Country',
 'Gender',
 'Demographics Question',
 'Demographics Response',
 'Question',
 'Survey Year',
 'Value']

In [243]:
#use your sales average to fill missing data. 
from pyspark.sql.functions import mean
#3.2 Dealing with the null value in 'Value' field
mean_value = df.select(mean(df['Value'])).collect()[0][0]
new_df = new_df.na.fill({'Value': mean_value})

In [244]:
new_df.show()
new_df.printSchema()

+--------+-----------+------+---------------------+---------------------+--------------------+-----------+-----------------+
|RecordID|    Country|Gender|Demographics Question|Demographics Response|            Question|Survey Year|            Value|
+--------+-----------+------+---------------------+---------------------+--------------------+-----------+-----------------+
|       1|Afghanistan|     F|       Marital status|        Never married|... if she burns ...| 01/01/2015|19.76253687315632|
|       1|Afghanistan|     F|            Education|               Higher|... if she burns ...| 01/01/2015|             10.1|
|       1|Afghanistan|     F|            Education|            Secondary|... if she burns ...| 01/01/2015|             13.7|
|       1|Afghanistan|     F|            Education|              Primary|... if she burns ...| 01/01/2015|             13.8|
|       1|Afghanistan|     F|       Marital status| Widowed, divorced...|... if she burns ...| 01/01/2015|             13.8|


In [245]:
new = 'Value'

new_df.select(new).show()
new_df.select(new).describe().show()

+-----------------+
|            Value|
+-----------------+
|19.76253687315632|
|             10.1|
|             13.7|
|             13.8|
|             13.8|
|             17.0|
|             17.3|
|             18.0|
|             18.1|
|             18.2|
|             18.3|
|             18.3|
|             18.8|
|             19.1|
|             20.8|
|19.76253687315632|
|              4.5|
|              4.6|
|              5.2|
|              6.3|
+-----------------+
only showing top 20 rows

+-------+------------------+
|summary|             Value|
+-------+------------------+
|  count|             12600|
|   mean| 19.76253687315705|
| stddev|16.005588776096097|
|    min|               0.0|
|    max|              86.9|
+-------+------------------+



In [246]:
#3.3 Constructing new data from 'Value' called 'Measurement'
from pyspark.sql.functions import mean, col, when 
data_copy_33 = new_df.withColumn(
    'Measurement',
    when(col('Value') == mean_value, 0).otherwise(1)
)


In [247]:
new = 'Measurement'

data_copy_33.select(new).show()
data_copy_33.select(new).describe().show()

+-----------+
|Measurement|
+-----------+
|          0|
|          1|
|          1|
|          1|
|          1|
|          1|
|          1|
|          1|
|          1|
|          1|
|          1|
|          1|
|          1|
|          1|
|          1|
|          0|
|          1|
|          1|
|          1|
|          1|
+-----------+
only showing top 20 rows

+-------+-------------------+
|summary|        Measurement|
+-------+-------------------+
|  count|              12600|
|   mean| 0.8878571428571429|
| stddev|0.31555465373180014|
|    min|                  0|
|    max|                  1|
+-------+-------------------+



In [248]:
data_copy_33.printSchema()

root
 |-- RecordID: integer (nullable = true)
 |-- Country: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Demographics Question: string (nullable = true)
 |-- Demographics Response: string (nullable = true)
 |-- Question: string (nullable = true)
 |-- Survey Year: string (nullable = true)
 |-- Value: double (nullable = false)
 |-- Measurement: integer (nullable = false)



In [249]:
#3.4 Merging with other dataframe
file2 = 'violence_data_merge.csv'
merge_df = spark.read.csv(file2, header=True, inferSchema=True)

In [250]:
merge_df.show()

+--------+----+----+----+----+----+----+----+-------------+
|RecordID| _c1| _c2| _c3| _c4| _c5| _c6| _c7|User feedback|
+--------+----+----+----+----+----+----+----+-------------+
|       1|null|null|null|null|null|null|null|         null|
|       1|null|null|null|null|null|null|null|         null|
|       1|null|null|null|null|null|null|null|         null|
|       1|null|null|null|null|null|null|null|           10|
|       1|null|null|null|null|null|null|null|         null|
|       1|null|null|null|null|null|null|null|         null|
|       1|null|null|null|null|null|null|null|           23|
|       1|null|null|null|null|null|null|null|            e|
|       1|null|null|null|null|null|null|null|           4t|
|       1|null|null|null|null|null|null|null|             |
|       1|null|null|null|null|null|null|null|         null|
|       1|null|null|null|null|null|null|null|         null|
|       1|null|null|null|null|null|null|null|          532|
|       1|null|null|null|null|null|null|

23/10/11 12:54:40 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: RecordID, , , , , , , , User feedback
 Schema: RecordID, _c1, _c2, _c3, _c4, _c5, _c6, _c7, User feedback
Expected: _c1 but found: 
CSV file: file:///home/ubuntu/722_I4/violence_data_merge.csv


In [251]:
merged_data = data_copy_33.join(merge_df, 'RecordID')

In [252]:
merged_data.show()
merged_data.printSchema()

23/10/11 12:54:40 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: RecordID, , , , , , , , User feedback
 Schema: RecordID, _c1, _c2, _c3, _c4, _c5, _c6, _c7, User feedback
Expected: _c1 but found: 
CSV file: file:///home/ubuntu/722_I4/violence_data_merge.csv


+--------+-----------+------+---------------------+---------------------+--------------------+-----------+-----------------+-----------+----+----+----+----+----+----+----+-------------+
|RecordID|    Country|Gender|Demographics Question|Demographics Response|            Question|Survey Year|            Value|Measurement| _c1| _c2| _c3| _c4| _c5| _c6| _c7|User feedback|
+--------+-----------+------+---------------------+---------------------+--------------------+-----------+-----------------+-----------+----+----+----+----+----+----+----+-------------+
|       1|Afghanistan|     F|       Marital status|        Never married|... if she burns ...| 01/01/2015|19.76253687315632|          0|null|null|null|null|null|null|null|           4e|
|       1|Afghanistan|     F|       Marital status|        Never married|... if she burns ...| 01/01/2015|19.76253687315632|          0|null|null|null|null|null|null|null|         gf43|
|       1|Afghanistan|     F|       Marital status|        Never marri

In [253]:
#3.5 Removing all the other fields that do not have anything contains
columns_to_drop = ['_c1', '_c2', '_c3', '_c4', '_c5', '_c6', '_c7']
merged_data = merged_data.drop(*columns_to_drop)

In [254]:
merged_data.printSchema()
new = 'Value'

df.select(new).show()
merged_data.select(new).describe().show()

root
 |-- RecordID: integer (nullable = true)
 |-- Country: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Demographics Question: string (nullable = true)
 |-- Demographics Response: string (nullable = true)
 |-- Question: string (nullable = true)
 |-- Survey Year: string (nullable = true)
 |-- Value: double (nullable = false)
 |-- Measurement: integer (nullable = false)
 |-- User feedback: string (nullable = true)

+-----+
|Value|
+-----+
| null|
| 10.1|
| 13.7|
| 13.8|
| 13.8|
| 17.0|
| 17.3|
| 18.0|
| 18.1|
| 18.2|
| 18.3|
| 18.3|
| 18.8|
| 19.1|
| 20.8|
| null|
|  4.5|
|  4.6|
|  5.2|
|  6.3|
+-----+
only showing top 20 rows

+-------+------------------+
|summary|             Value|
+-------+------------------+
|  count|            378000|
|   mean| 19.76253687315055|
| stddev|16.004974791697258|
|    min|               0.0|
|    max|              86.9|
+-------+------------------+



In [255]:
#4.1 Feature selection & Encoding the string fields
from pyspark.ml.feature import StringIndexer, OneHotEncoder
from pyspark.ml.feature import VectorAssembler

In [256]:
#Encoding 'Country'
indexer = StringIndexer(inputCol="Country", outputCol="CountryIndex")
encoder = OneHotEncoder(inputCols=["CountryIndex"], outputCols=["CountryVec"])
df = indexer.fit(df).transform(df)
df = encoder.fit(df).transform(df)

In [257]:
df.printSchema()

root
 |-- RecordID: integer (nullable = true)
 |-- Country: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Demographics Question: string (nullable = true)
 |-- Demographics Response: string (nullable = true)
 |-- Question: string (nullable = true)
 |-- Survey Year: string (nullable = true)
 |-- Value: double (nullable = true)
 |-- User feedback: string (nullable = true)
 |-- CountryIndex: double (nullable = false)
 |-- CountryVec: vector (nullable = true)



In [258]:
#Encoding 'Gender'
indexer = StringIndexer(inputCol="Gender", outputCol="GenderIndex")
encoder = OneHotEncoder(inputCols=["GenderIndex"], outputCols=["GenderVec"])
df = indexer.fit(df).transform(df)
df = encoder.fit(df).transform(df)
df.printSchema()

root
 |-- RecordID: integer (nullable = true)
 |-- Country: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Demographics Question: string (nullable = true)
 |-- Demographics Response: string (nullable = true)
 |-- Question: string (nullable = true)
 |-- Survey Year: string (nullable = true)
 |-- Value: double (nullable = true)
 |-- User feedback: string (nullable = true)
 |-- CountryIndex: double (nullable = false)
 |-- CountryVec: vector (nullable = true)
 |-- GenderIndex: double (nullable = false)
 |-- GenderVec: vector (nullable = true)



In [259]:
#Encoding 'Demographics Response' 
#responseIndexer = StringIndexer(inputCol="Demographics Response", outputCol="ResponseIndex")
#responseEncoder = OneHotEncoder(inputCols=["ResponseIndex"], outputCols=["ResponseVec"])
#df = responseIndexer.fit(df).transform(df)
#df = responseEncoder.fit(df).transform(df)
indexer = StringIndexer(inputCol="Demographics Response", outputCol="ResponseIndex")
df = indexer.fit(df).transform(df)
df.printSchema()

root
 |-- RecordID: integer (nullable = true)
 |-- Country: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Demographics Question: string (nullable = true)
 |-- Demographics Response: string (nullable = true)
 |-- Question: string (nullable = true)
 |-- Survey Year: string (nullable = true)
 |-- Value: double (nullable = true)
 |-- User feedback: string (nullable = true)
 |-- CountryIndex: double (nullable = false)
 |-- CountryVec: vector (nullable = true)
 |-- GenderIndex: double (nullable = false)
 |-- GenderVec: vector (nullable = true)
 |-- ResponseIndex: double (nullable = false)



In [260]:
df.show()
#retreat

new_df = df.na.fill(0)


+--------+-----------+------+---------------------+---------------------+--------------------+-----------+-----+-------------+------------+--------------+-----------+-------------+-------------+
|RecordID|    Country|Gender|Demographics Question|Demographics Response|            Question|Survey Year|Value|User feedback|CountryIndex|    CountryVec|GenderIndex|    GenderVec|ResponseIndex|
+--------+-----------+------+---------------------+---------------------+--------------------+-----------+-----+-------------+------------+--------------+-----------+-------------+-------------+
|       1|Afghanistan|     F|       Marital status|        Never married|... if she burns ...| 01/01/2015| null|         null|         0.0|(69,[0],[1.0])|        0.0|(1,[0],[1.0])|          7.0|
|       1|Afghanistan|     F|            Education|               Higher|... if she burns ...| 01/01/2015| 10.1|         null|         0.0|(69,[0],[1.0])|        0.0|(1,[0],[1.0])|          5.0|
|       1|Afghanistan|   

In [261]:
new_df.show()

+--------+-----------+------+---------------------+---------------------+--------------------+-----------+-----+-------------+------------+--------------+-----------+-------------+-------------+
|RecordID|    Country|Gender|Demographics Question|Demographics Response|            Question|Survey Year|Value|User feedback|CountryIndex|    CountryVec|GenderIndex|    GenderVec|ResponseIndex|
+--------+-----------+------+---------------------+---------------------+--------------------+-----------+-----+-------------+------------+--------------+-----------+-------------+-------------+
|       1|Afghanistan|     F|       Marital status|        Never married|... if she burns ...| 01/01/2015|  0.0|         null|         0.0|(69,[0],[1.0])|        0.0|(1,[0],[1.0])|          7.0|
|       1|Afghanistan|     F|            Education|               Higher|... if she burns ...| 01/01/2015| 10.1|         null|         0.0|(69,[0],[1.0])|        0.0|(1,[0],[1.0])|          5.0|
|       1|Afghanistan|   

In [262]:
from pyspark.sql.functions import skewness
skewness_value = new_df.select(skewness(new_df['Value'])).collect()[0][0]
print(f"Skewness of 'Value': {skewness_value}")


Skewness of 'Value': 1.1445442844796359


In [263]:
from pyspark.sql.functions import sqrt
new_df = new_df.withColumn('new_values', sqrt(new_df['Value']))


In [264]:
new_df.show()
new_df.printSchema()

+--------+-----------+------+---------------------+---------------------+--------------------+-----------+-----+-------------+------------+--------------+-----------+-------------+-------------+------------------+
|RecordID|    Country|Gender|Demographics Question|Demographics Response|            Question|Survey Year|Value|User feedback|CountryIndex|    CountryVec|GenderIndex|    GenderVec|ResponseIndex|        new_values|
+--------+-----------+------+---------------------+---------------------+--------------------+-----------+-----+-------------+------------+--------------+-----------+-------------+-------------+------------------+
|       1|Afghanistan|     F|       Marital status|        Never married|... if she burns ...| 01/01/2015|  0.0|         null|         0.0|(69,[0],[1.0])|        0.0|(1,[0],[1.0])|          7.0|               0.0|
|       1|Afghanistan|     F|            Education|               Higher|... if she burns ...| 01/01/2015| 10.1|         null|         0.0|(69,[

In [265]:
###########################################################

In [266]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [267]:
# The input columns are the feature column names, and the output column is what you'd like the new column to be named. 
assembler = VectorAssembler(
    inputCols=["CountryVec","ResponseIndex"],
    outputCol="features")

In [268]:
# Now that we've created the assembler variable, let's actually transform the data.
output = assembler.transform(new_df)

In [269]:
#6.1 Linear Regression
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator

# driven the training and test data
train_data, test_data = output.randomSplit([0.7, 0.3])

# state the linear regression model
lr = LogisticRegression(featuresCol='features', labelCol='GenderIndex')

# fit the linear regression model
lr_model = lr.fit(train_data)

# find the output of the coeficients
print(f"Coefficients: {lr_model.coefficients}, Intercept: {lr_model.intercept}")




Coefficients: [0.01522306738223122,0.0298552977226573,-0.017236894003873793,0.028036434739986484,-0.004414144898297213,-0.02870008100406631,-0.017206057530807003,0.07820131667469385,0.029749828065116904,0.24592067490050545,-0.04560013062641783,0.16102236806832657,0.15718879102973696,0.02870852869031936,0.07734616376005686,0.02895602212314128,0.09184116649019879,-0.0863911531350849,0.07549986102990142,0.09572532057585569,0.061590114959301286,0.013040389627662257,0.22179779466925412,0.10591435601890323,0.06146714192064532,0.015814593046079392,0.0013920510285786556,-0.053041730502864017,-0.031055470403283315,0.18920987525130054,-0.03504238289409379,0.07757950918374887,0.012987357658354537,-0.10877561418463079,0.06153424080075311,0.18495513855080709,0.15197631177083112,-0.14276982345785363,-0.034605956780609404,-0.14285696582464438,0.1438413122165253,0.14376592570555213,0.030481187105559658,0.1650346928550842,0.02843061519167639,-0.06854171741527292,0.18076015683797592,-0.03350760745741622

In [270]:
# transform the predict result 
pred_results = lr_model.transform(test_data)

# use the show() function to show the predict result 
pred_results.show()



+--------+-----------+------+---------------------+---------------------+--------------------+-----------+-----+-------------+------------+--------------+-----------+-------------+-------------+-------------------+--------------------+--------------------+--------------------+----------+
|RecordID|    Country|Gender|Demographics Question|Demographics Response|            Question|Survey Year|Value|User feedback|CountryIndex|    CountryVec|GenderIndex|    GenderVec|ResponseIndex|         new_values|            features|       rawPrediction|         probability|prediction|
+--------+-----------+------+---------------------+---------------------+--------------------+-----------+-----+-------------+------------+--------------+-----------+-------------+-------------+-------------------+--------------------+--------------------+--------------------+----------+
|       1|Afghanistan|     F|            Education|            Secondary|... if she burns ...| 01/01/2015| 13.7|         null|       

In [272]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
#Caculate the accurancy of the model
# By using the MulticlassClassificationEvaluator 
evaluator = MulticlassClassificationEvaluator(
    labelCol="GenderIndex", predictionCol="prediction", metricName="accuracy")

# find and print the accurancy
accuracy = evaluator.evaluate(pred_results)
print(f"Accuracy: {accuracy}")

Accuracy: 0.45805942676834077
